<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 15px; height: 80px">

# Project 6:  Web Scraping
### Finding Underpriced RVs on Craigslist

![](https://snag.gy/WrdUMx.jpg)

In this project we will be practicing our web scraping skills.  You can use Scrapy or Python requests in order to complete this project.  It may be helpful to write some prototype code in this notebook to test your assumptions, then move it into a Python file that can be run from the command line.

> In order to run code from the command line, instead of the notebook, you just need to save your code to a file (with a .py extension), and run it using the Python interpreter:<br><br>
> `python my_file.py`

You will be building a process to scrape a single category of search results on Craigslist, that can easily be applied to other categories by changing the search terms.  The main goal is to be able to target and scrape a single page given a set of parameters.

**If you use Scrapy, provide your code in a folder.**

## Import your libraries for scrapy / requests / pandas / numpy / etc
Setup whichever libraries you need. Review past material for reference.

In [84]:
# PREPARE REQUIRED LIBRARIES OK
#Import
import requests
import random
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.  Scrape for the largest US cities (non-exhaustive list)
Search, research, and scrape Wikipedia for a list of the largest US cities.  There are a few sources but find one that is in a nice table.  We don't want all cities, just signifficant cities.  Examine your source.  Look for what can be differentiable.

- Use requests
- Build XPath query(ies)
- Extract to a list
- Clean your list

In [85]:
# SCRAPE WIKIPEDIA FOR LARGEST US CITIES (NON-EXHAUSTIVE LIST)
#Trytominimizehitstosite,soisolate
response=requests.get("http://www.infoplease.com/ipa/a0763098.html")

In [86]:
HTML=response.text
population=Selector(text=HTML).xpath('//tr/td[3]/text()').extract()
cities=Selector(text=HTML).xpath('//tr/td[2]/a/text()').extract()
print cities[0:5]
a=pd.Series(cities,name='city')
b=pd.Series(population,name='population')
combined=pd.concat([a,b],axis=1)
combined['city']=combined['city'].map(lambda x: x.replace(',',''))
combined['population']=combined['population'].map(lambda x: int(x.replace(',','')))
combined.head()

[u'New York', u'Los Angeles', u'Chicago', u'Houston', u'Philadelphia']


,city,population
0,New York,8491079
1,Los Angeles,3928864
2,Chicago,2722389
3,Houston,2239558
4,Philadelphia,1560297


In [87]:
random.choice(combined['city']) #chooseLongBeach

u'El Paso'

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 1.2 Only retain cities with properly formed ASCII

Optionally, filter out any cities with impropper ASCII characters.  A smaller list will be easier to look at.  However you may not need to filter these if you spend more time scraping a more concise city list.  This list should help you narrow down the list of regional Craigslist sites.

In [88]:
# ONLY RETAIN PROPERLY FORMED CITIES WITH FILTERING FUNCTION


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.  Write a function to capture current pricing information via Craigslist in one city.
Choose a city from your scraped data, then go to the cooresponding city section on Craigslist, searching for "rv" in the auto section.  Write a method that pulls out the prices.

In [89]:

#population=Selector(text=HTML).xpath('//tr/td[3]/text()').extract()
#response=requests.get("https://www.craigslist.org/about/sites")
class RV(object):
    def __init__(self):
        self.URL = ''     
    
#     def fetch_region(self,x):
#         return region_dict[x]


    def fetch_prices(self,reg):
        rv_prices = []
        URL = "https://" + reg + ".craigslist.org/search/rva"
        response=requests.get(URL)
        print response.status_code
        if response.status_code == 200:
            HTML=response.text
            try:
                rv_prices=Selector(text=HTML).xpath('//span/span/span[@class="price"]/text()').extract()
            except:
                rv_prices = []
#         for x in range(0,len(rv_prices)):
#             rv_prices[x] = int(rv_prices[x].replace("$",''))
# #            rv = pd.DataFrame(population,columns=['prices'])
#            return np.mean(rv)
#             rv_ret.append(np.median(rv))
#             rv_ret[2].append(rv_prices

        return rv_prices
        
    
    
a = RV()
z = a.fetch_prices('dallas')
print z[0:5]


200
[u'$23000', u'$500', u'$34444', u'$800', u'$21595']


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 2.1 Create a mapping of cities to cooresponding regional Craigslist URLs

Major US cities on Craigslist typically have their own cooresponding section (ie: SFBay Area, NYC, Boston, Miami, Seattle, etc).  Later, you will use these to query search results for various metropolitian regions listed on Craigslist.  Between the major metropolitan Craigslist sites, the only thing that will differ is the URL's that correspond to them.

The point of the "mapping":  Create a data structure that allows you to iterate with both the name of the city from Wikipedia, with the cooresponding variable that that will allow you to construct each craigslist URL for each region.

> For San Francsico (the Bay Area metropolitan area), the url for the RV search result is:
> http://sfbay.craigslist.org/search/sss?query=rv
>
> The convention is http://[region].craigslist.org/search/sss?query=rf
> Replacing [region] with the cooresponding city name will allow you to quickly iterate through each regional Craigslist site, and scrape the prices from the search results.  Keep this in mind while you build this "mapping".


In [90]:
# Need to get craigslists regional names for a data dictionary
#Try to minimize hits to site,so isolate
response=requests.get("https://www.craigslist.org/about/sites")

In [91]:
HTML=response.text
craigslist_city_names=Selector(text=HTML).xpath('//li/a/text()').extract()
craigslist_city_names

[u'auburn',
 u'birmingham',
 u'dothan',
 u'florence / muscle shoals',
 u'gadsden-anniston',
 u'huntsville / decatur',
 u'mobile',
 u'montgomery',
 u'tuscaloosa',
 u'anchorage / mat-su',
 u'fairbanks',
 u'kenai peninsula',
 u'southeast alaska',
 u'flagstaff / sedona',
 u'mohave county',
 u'phoenix',
 u'prescott',
 u'show low',
 u'sierra vista',
 u'tucson',
 u'yuma',
 u'fayetteville ',
 u'fort smith',
 u'jonesboro',
 u'little rock',
 u'texarkana',
 u'bakersfield',
 u'chico',
 u'fresno / madera',
 u'gold country',
 u'hanford-corcoran',
 u'humboldt county',
 u'imperial county',
 u'inland empire',
 u'los angeles',
 u'mendocino county',
 u'merced',
 u'modesto',
 u'monterey bay',
 u'orange county',
 u'palm springs',
 u'redding',
 u'sacramento',
 u'san diego',
 u'san francisco bay area',
 u'san luis obispo',
 u'santa barbara',
 u'santa maria',
 u'siskiyou county',
 u'stockton',
 u'susanville',
 u'ventura county',
 u'visalia-tulare',
 u'yuba-sutter',
 u'boulder',
 u'colorado springs',
 u'denver

In [92]:
region_dict={}
for city in combined['city']:
    Found=False
    for region in craigslist_city_names:
        if city.lower()in region:
            region_dict[city]=region.replace(' ','')
            Found=True
            break
        else:
            region_dict[city]="NotFound"
for region in region_dict:
    if region_dict[region]=="NotFound":
        print region

Arlington
Washington DC
Nashville-Davidson
Virginia Beach
Mesa
San Jose
Louisville-Jefferson County
Oakland
Long Beach


In [93]:
region_dict['Nashville']="nashville"
region_dict['VirginiaBeach']="richmond"
region_dict['Washington,DC']="washington"
region_dict['Louisville']="louisville"
region_dict['San Francisco']="sanfrancisco"
region_dict['New York']="newyork"
region_dict['Colorado Springs']="cosprings"
region_dict['Omaha']="omaha"
region_dict['Miami']="miami"
region_dict['Dallas']="dallas"
region_dict['Columbus']="Columbus"
region_dict['Albuquerque']="Albuquerque"
region_dict['Detroit']="detroit"
region_dict['Raleigh']="raleigh"
region_dict['Fresno']="fresno"
region_dict['Minneapolis']="minneapolis"
region_dict['Phoenix']="phoenix"
region_dict['Seattle']="seattle"
region_dict['Fort Worth']="NotFound"

del region_dict['Nashville-Davidson']
del region_dict['Louisville-Jefferson County']
#Dictcomprehension

region_dict={k:v for k,v in region_dict.iteritems() if v!='NotFound'}

region_dict

{u'Albuquerque': 'Albuquerque',
 u'Atlanta': u'atlanta',
 u'Austin': u'austin',
 u'Baltimore': u'baltimore',
 u'Boston': u'boston',
 u'Charlotte': u'charlotte',
 u'Chicago': u'chicago',
 u'Cleveland': u'cleveland',
 u'Colorado Springs': 'cosprings',
 u'Columbus': 'Columbus',
 u'Dallas': 'dallas',
 u'Denver': u'denver',
 u'Detroit': 'detroit',
 u'El Paso': u'elpaso',
 u'Fresno': 'fresno',
 u'Houston': u'houston',
 u'Indianapolis': u'indianapolis',
 u'Jacksonville': u'jacksonville',
 u'Kansas City': u'kansascity',
 u'Las Vegas': u'lasvegas',
 u'Los Angeles': u'losangeles',
 'Louisville': 'louisville',
 u'Memphis': u'memphis',
 u'Miami': 'miami',
 u'Milwaukee': u'milwaukee',
 u'Minneapolis': 'minneapolis',
 'Nashville': 'nashville',
 u'New Orleans': u'neworleans',
 u'New York': 'newyork',
 u'Oklahoma City': u'oklahomacity',
 u'Omaha': 'omaha',
 u'Philadelphia': u'philadelphia',
 u'Phoenix': 'phoenix',
 u'Portland': u'portland',
 u'Raleigh': 'raleigh',
 u'Sacramento': u'sacramento',
 u'San

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 3. Define a function to caculate mean and median price per city.

Now that you've created a list of cities you want to scrape, adapt your solution for grabbing data in one region site, to grab data for all regional sites that you collected, then calculate the mean and median price of RV results from each city.

> Look at the URLs from a few different regions (ie: portland, phoenix, sfbay), and find what they have in common.  Determine the area in the URL string that needs to change the least, and figure out how to replace only that portion of the URL in order to iterate through each city.

In [94]:
#Adapting solution to multiple sites in anticipation of the mean / median work

collect_dict = {}
for key in region_dict:
    a = RV() #class
    value = region_dict[key]
    print key,value
    if key not in []:
        z = a.fetch_prices(value)
        collect_dict[key] = z
        print z[0:5]
    
#     'Dallas','Columbus','Albuquerque','Miami',
#                     'Detroit','Raleigh','Fresno','Minneapolis'
#                      ,'Phoenix','Seattle'


San Diego sandiego
200
[u'$29990', u'$24500', u'$18500', u'$78000', u'$20000']
Dallas dallas
200
[u'$23000', u'$500', u'$34444', u'$800', u'$21595']
Philadelphia philadelphia
200
[u'$35000', u'$11900', u'$39000', u'$14900', u'$24900']
Denver denver
200
[u'$15900', u'$10200', u'$1500', u'$15900', u'$15000']
Louisville louisville
200
[u'$50', u'$1500', u'$17500', u'$1500', u'$2500']
Washington,DC washington
200
[u'$14426', u'$3500', u'$2000', u'$10000', u'$7300']
San Francisco sanfrancisco
200
[u'$39900', u'$49988', u'$56990', u'$2995', u'$1500']
Columbus Columbus
200
[u'$8', u'$16900', u'$14995', u'$7500', u'$4200']
Indianapolis indianapolis
200
[u'$2900', u'$22995', u'$20995', u'$1500', u'$16995']
Phoenix phoenix
200
[u'$55000', u'$2900', u'$5500', u'$9500', u'$2400']
Sacramento sacramento
200
[u'$32000', u'$1400', u'$7000', u'$39990', u'$6200']
Nashville nashville
200
[u'$69900', u'$7100', u'$21900', u'$250000', u'$19900']
Albuquerque Albuquerque
200
[u'$1500', u'$1500', u'$22999', u'

In [95]:
frames = []
for keyrv in collect_dict:
    region_list = collect_dict[keyrv]
    clean = []
    clean = [float(x.replace("$","")) for x in region_list if len(x)>0]
    clean = filter(lambda a: a >= 9999.0, clean)
    df = pd.DataFrame(clean,columns=["rv_price"])
    df['city']= df["rv_price"].map(lambda x: keyrv)
    frames.append(df)
    
result = pd.concat(frames)
result


#pd.DataFrame(rows, columns=["State", "R", "D", "incumbent"])

,rv_price,city
0,69900.0,Nashville
1,21900.0,Nashville
2,250000.0,Nashville
3,19900.0,Nashville
4,16500.0,Nashville
5,26900.0,Nashville
6,49900.0,Nashville
7,26900.0,Nashville
8,14500.0,Nashville
9,14900.0,Nashville


In [96]:
rv_mean = result.groupby(['city']).mean()
rv_mean = rv_mean.reset_index()
rv_mean.rename(columns={'rv_price':'rv_mean'}, inplace=True)
rv_mean

,city,rv_mean
0,Albuquerque,56656.347222
1,Atlanta,27781.172414
2,Austin,49541.508772
3,Baltimore,26476.828571
4,Boston,37773.555556
5,Charlotte,27750.000000
6,Chicago,22557.651163
7,Cleveland,36130.941176
8,Colorado Springs,36510.255319
9,Columbus,31817.191489


In [97]:
rv_median = result.groupby(['city']).median()
rv_median = rv_median.reset_index()
rv_median.rename(columns={'rv_price':'rv_median'}, inplace=True)
rv_median

,city,rv_median
0,Albuquerque,49995.0
1,Atlanta,21995.0
2,Austin,33988.0
3,Baltimore,16950.0
4,Boston,24999.0
5,Charlotte,23500.0
6,Chicago,18500.0
7,Cleveland,27997.5
8,Colorado Springs,24900.0
9,Columbus,15900.0


<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 4. Run your scraping process, and save your results to a CSV file.

In [98]:
loot = "/Users/paulmartin/Desktop/rv.csv"
df.to_csv(loot, sep='\t')

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 5. Do an analysis of the RV market.

Go head we'll wait.  Anything notable about the data?

In [99]:
result = pd.merge(rv_mean,rv_median,how='left', on=['city'])
result['difference'] = result['rv_mean']-result['rv_median']
result.sort_values('rv_mean', axis=0, ascending=False, inplace=True, kind='quicksort', na_position='last')
result
#note. To eliminate some ofthe noise rv's undesr 10K were excluded

,city,rv_mean,rv_median,difference
13,El Paso,71100.287500,64995.0,6105.287500
19,Las Vegas,67029.855263,32050.0,34979.855263
17,Jacksonville,63810.444444,30447.5,33362.944444
29,Oklahoma City,59332.019608,49995.0,9337.019608
0,Albuquerque,56656.347222,49995.0,6661.347222
15,Houston,56068.421053,35000.0,21068.421053
23,Miami,55881.236364,41500.0,14381.236364
40,Tucson,54897.718750,45495.0,9402.718750
20,Los Angeles,51116.500000,42900.0,8216.500000
32,Phoenix,50911.078431,30349.0,20562.078431


In [100]:
from geopy import Nominatim
from geopy.distance import vincenty

In [101]:
def getmiles(a,b):
    geolocator = Nominatim()
    location1 = geolocator.geocode(a)
    location2 = geolocator.geocode(b)
    return (vincenty((location1.latitude, location1.longitude), (location2.latitude, location2.longitude)).miles)
    

In [102]:
print getmiles( "san diego","newyork")

2435.55757769


In [120]:
from itertools import product
l1 = result.city.tolist()
z = list(product(l1, l1))
miles_davis = pd.DataFrame(z,columns=['city_beg','city_end'])
miles_davis['miles'] = miles_davis['city_beg','city_end'].map(lambda x,y:getmiles(x,y))



KeyError: ('city_beg', 'city_end')

<img src="http://imgur.com/l5NasQj.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.1 Does it makes sense to buy RVs in one region and sell them in another?

Assuming the cost of shipping or driving from one regional market to another.

In [104]:
#see below

this one is a bit hard to answer given some of the feature blind spots. There is variability within markets chich is generally positively skewed.  That represents some market to market arbitrage potentially.  There are also some intra regional mean differences which would suggest some opportunity (See Washington DC to Baltimore).


<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.2 Can you pull out the "make" from the markup and include that in your analyis?
How reliable is this data and does it make sense?

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

### 5.3 Are there any other variables you could pull out of the markup to help describe your dataset?

In [105]:
#mileage, condition, year

<img src="http://imgur.com/xDpSobf.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 6. Move your project into scrapy (if you haven't used Scrapy yet)

>Start a project by using the command `scrapy startproject [projectname]`
> - Update your settings.py (review our past example)
> - Update your items.py
> - Create a spiders file in your `[project_name]/[project_name]/spiders` directory

You can update your spider class with the complete list of craigslist "start urls" to effectively scrape all of the regions.  Start with one to test.

Updating your parse method with the method you chose should require minimal changes.  It will require you to update your parse method to use the response parameter, and an item model (defined in items.py).

<img src="http://imgur.com/GCAf1UX.png" style="float: left; margin: 25px 15px 0px 0px; height: 25px">

## 7.  Chose another area of Craigslist to scrape.

**Choose an area having more than a single page of results, then scrape multiple regions, multiple pages of search results and or details pages.**

This is the true exercise of being able to understand how to succesffuly plan, develop, and employ a broader scraping strategy.  Even though this seems like a challenging task, a few tweeks of your current code can make this very managable if you've pieced together all the touch points.  If you are still confused as to some of the milestones within this process, this is an excellent opportunity to round out your understanding, or help you build a list of questions to fill in your gaps.

_Use Scrapy!  Provide your code in this project directory when you submit this project._